In [2]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
file_path = "../data/raw/raw_analyst_ratings.csv"

df = pd.read_csv(file_path)
df

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A
...,...,...,...,...,...,...
1407323,1413844,Top Narrow Based Indexes For August 29,https://www.benzinga.com/news/11/08/1888782/to...,Monica Gerson,2011-08-29 00:00:00,ZX
1407324,1413845,Recap: Wednesday's Top Percentage Gainers and ...,https://www.benzinga.com/news/earnings/11/06/1...,Benjamin Lee,2011-06-22 00:00:00,ZX
1407325,1413846,UPDATE: Oppenheimer Color on China Zenix Auto ...,https://www.benzinga.com/analyst-ratings/analy...,BenzingaStaffL,2011-06-21 00:00:00,ZX
1407326,1413847,Oppenheimer Initiates China Zenix At Outperfor...,https://www.benzinga.com/analyst-ratings/price...,Joe Young,2011-06-21 00:00:00,ZX


In [4]:
df["date"] = pd.to_datetime(df["date"], format="ISO8601", utc=True)
df["date_only"] = df["date"].dt.date
df["time_only"] = df["date"].dt.time
df

,Unnamed: 0,headline,url,publisher,date,stock,date_only,time_only
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 14:30:54+00:00,A,2020-06-05,14:30:54
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 14:45:20+00:00,A,2020-06-03,14:45:20
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 08:30:07+00:00,A,2020-05-26,08:30:07
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 16:45:06+00:00,A,2020-05-22,16:45:06
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 15:38:59+00:00,A,2020-05-22,15:38:59
...,...,...,...,...,...,...,...,...
1407323,1413844,Top Narrow Based Indexes For August 29,https://www.benzinga.com/news/11/08/1888782/to...,Monica Gerson,2011-08-29 04:00:00+00:00,ZX,2011-08-29,04:00:00
1407324,1413845,Recap: Wednesday's Top Percentage Gainers and ...,https://www.benzinga.com/news/earnings/11/06/1...,Benjamin Lee,2011-06-22 04:00:00+00:00,ZX,2011-06-22,04:00:00
1407325,1413846,UPDATE: Oppenheimer Color on China Zenix Auto ...,https://www.benzinga.com/analyst-ratings/analy...,BenzingaStaffL,2011-06-21 04:00:00+00:00,ZX,2011-06-21,04:00:00
1407326,1413847,Oppenheimer Initiates China Zenix At Outperfor...,https://www.benzinga.com/analyst-ratings/price...,Joe Young,2011-06-21 04:00:00+00:00,ZX,2011-06-21,04:00:00


In [5]:
df["headline_length"] = df["headline"].apply(lambda x: len(x))
df.head()

,Unnamed: 0,headline,url,publisher,date,stock,date_only,time_only,headline_length
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 14:30:54+00:00,A,2020-06-05,14:30:54,39
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 14:45:20+00:00,A,2020-06-03,14:45:20,42
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 08:30:07+00:00,A,2020-05-26,08:30:07,29
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 16:45:06+00:00,A,2020-05-22,16:45:06,44
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 15:38:59+00:00,A,2020-05-22,15:38:59,87


In [6]:
publishers = df["publisher"].value_counts()

# print("Maximal number of articles by a single publisher:", publishers.max())
max_publisher = publishers.max()

publishers[:10].plot(
    kind="bar", figsize=(10, 5), title="Top 10 publishers by number of articles"
)

plt.xlabel("Publisher")
plt.ylabel("Number of Articles Published by Publisher")


plt.xticks(rotation=45)
# plt.yticks(np.arange(1, max_publisher + 1))

([<matplotlib.axis.YTick at 0x7b0a7bbc7710>,
  ...],
 [Text(0, 1, '1'),
  Text(0, 2, '2'),
  Text(0, 3, '3'),
  Text(0, 4, '4'),
  Text(0, 5, '5'),
  Text(0, 6, '6'),
  Text(0, 7, '7'),
  Text(0, 8, '8'),
  Text(0, 9, '9'),
  Text(0, 10, '10'),
  Text(0, 11, '11'),
  Text(0, 12, '12'),
  Text(0, 13, '13'),
  Text(0, 14, '14'),
  Text(0, 15, '15'),
  Text(0, 16, '16'),
  Text(0, 17, '17'),
  Text(0, 18, '18'),
  Text(0, 19, '19'),
  Text(0, 20, '20'),
  Text(0, 21, '21'),
  Text(0, 22, '22'),
  Text(0, 23, '23'),
  Text(0, 24, '24'),
  Text(0, 25, '25'),
  Text(0, 26, '26'),
  Text(0, 27, '27'),
  Text(0, 28, '28'),
  Text(0, 29, '29'),
  Text(0, 30, '30'),
  Text(0, 31, '31'),
  Text(0, 32, '32'),
  Text(0, 33, '33'),
  Text(0, 34, '34'),
  Text(0, 35, '35'),
  Text(0, 36, '36'),
  Text(0, 37, '37'),
  Text(0, 38, '38'),
  Text(0, 39, '39'),
  Text(0, 40, '40'),
  Text(0, 41, '41'),
  Text(0, 42, '42'),
  Text(0, 43, '43'),
  Text(0, 44, '44'),
  Text(0, 45, '45'),
  Text(0, 46, '46'),

Error in callback <function _draw_all_if_interactive at 0x7b0a997b4540> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

: 

: 